In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device)
block_size = 8    #HYPERPARAMETER
batch_size = 4    #HYPERPARAMETER

mps


In [3]:
with open('wizard_of_oz.txt', 'r', encoding ='utf-8') as f:
    text = f.read()
print(len(text))

232313


In [4]:
type(text)

str

Printing the first 200 characters of the text:

In [2]:
print(text[:200])






  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC.


## Encoding and Decoding

In [4]:
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
print(len(chars))

80


In [6]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [7]:
encoded_hello = encode('hello')
encoded_hello

[61, 58, 65, 65, 68]

In [8]:
decoded_hello = decode(encoded_hello)
decoded_hello

'hello'

In [9]:
data = torch.tensor(encode(text), dtype = torch.long)

In [10]:
print(data[:100])

tensor([ 0,  0,  0,  0,  0,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,
         1, 44, 32, 29,  1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,
         0,  1,  1, 26, 49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1,
        26, 25, 45, 37,  0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1,
        44, 32, 29,  1, 47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1,
        44, 32, 29,  1, 36, 25, 38, 28,  1, 39])


## Training - Validation Split

In [11]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y
x, y = get_batch('train')
print('inputs:')
print(x.shape)
print(x)
print('targets:')
print(y)

tensor([12787, 92148, 51290, 73222])
inputs:
torch.Size([4, 8])
tensor([[58,  1, 54, 67, 62, 66, 54, 65],
        [68, 76,  1, 68, 67,  1, 54,  1],
        [66, 68, 71, 62, 54, 65, 11,  3],
        [54, 67, 57,  1, 76, 71, 62, 73]], device='mps:0')
targets:
tensor([[ 1, 54, 67, 62, 66, 54, 65,  1],
        [76,  1, 68, 67,  1, 54,  1, 65],
        [68, 71, 62, 54, 65, 11,  3,  0],
        [67, 57,  1, 76, 71, 62, 73, 61]], device='mps:0')


## Bigram

In [19]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([0]) target is tensor(0)
when input is tensor([0, 0]) target is tensor(0)
when input is tensor([0, 0, 0]) target is tensor(0)
when input is tensor([0, 0, 0, 0]) target is tensor(0)
when input is tensor([0, 0, 0, 0, 0]) target is tensor(1)
when input is tensor([0, 0, 0, 0, 0, 1]) target is tensor(1)
when input is tensor([0, 0, 0, 0, 0, 1, 1]) target is tensor(28)
when input is tensor([ 0,  0,  0,  0,  0,  1,  1, 28]) target is tensor(39)


In [ ]:
class Bigra